In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
from scipy.ndimage.measurements import label
from math import sin, cos, sqrt, atan2, radians
import scipy.stats as st
import geopy.distance
import requests

import metpy.calc as mpcalc
import metpy.constants as mpconstants
from metpy.units import units
from siphon.catalog import TDSCatalog
from siphon import catalog, ncss
from siphon.ncss import NCSS
from netCDF4 import Dataset
import datetime

%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir_rain='/thorncroftlab_rit/ahenny/rain/tccip_daily/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'
#this script plot synoptic variables on specified days

In [2]:
#Retrieve login credintial for RDA.
from pathlib import Path
HOME = str(Path.home())
credFile = open(HOME+'/.rdarc','r')
userId, pw = credFile.read().split()
session = requests.Session()
session.auth = (userId, pw)

In [ ]:
ds=xr.open_dataset(dir+'ls_extreme_rain_taiwan.nc')
p=ds['large_scale_extreme_rain_all']#total rainfall on ER days
threshold=ds['threshold_99']
dates=ds.large_scale_extreme_days
print(dates)

ds1=xr.open_dataset(dir+'ivt_basics_taiwan_95.nc')
ivt_threshold=ds1['ivt_threshold']

ds4=xr.open_dataset(dir+'taiwan_ep_days_stats_var95_newar.nc')
ar_yesno4=ds4['ar_yesno'].values.tolist()
ivt_yesno4=ds4['ivt_yesno'].values.tolist()
tc_yesno4=ds4['tc_yesno'].values.tolist()
other_yesno4=ds4['other_yesno'].values.tolist()
tc_linked_ar_yesno4=ds4['tc_linked_ar_yesno'].values.tolist()
tc_linked_ivt_yesno4=ds4['tc_linked_ivt_yesno'].values.tolist()
tc_remnant_linked_ivt_yesno4=ds4['ivt_tc_remnants_yesno'].values.tolist()
tc_remnant_ar_combo_yesno4=ds4['ar_tc_remnant_combo_yesno'].values.tolist()
tc_remnant_linked_ar_yesno4=ds4['tc_remnant_linked_ar_yesno'].values.tolist()
tc_remnants_yesno4=ds4['tc_remnants_yesno'].values.tolist()
tc_ar_combo_yesno4=ds4['tc_ar_combo_yesno'].values.tolist()

ds6=xr.open_dataset(dir+'IBTrACS.WP.v04r00.nc')
#print(ds)
lat_tc=ds6.lat
lon_tc=ds6.lon
#nature_tc=ds6.nature
ds7=xr.open_dataset(dir+'ibtracs_wp_time.nc')
years_tc=ds7.years
months_tc=ds7.months
days_tc=ds7.days
hours_tc=ds7.hours
nature_tc=ds7.nature#1 if tropical, 0 if not

ds5=xr.open_dataset(dir+'taiwan_ar_climo_mys.nc')
print(ds5)
ar=ds5.ar
tc_trop=ds5.tc_trop
tc_nontrop=ds5.tc_nontrop
labeled=ds5['labeled']

df=xr.open_dataset(dir+'era_5_mys_mslp_taiwan.nc')
slp=df.msl

zipped_tc=list(zip(dates,tc_yesno4))
print([x for x in zipped_tc if x[1]==1])

In [4]:
df1=xr.open_dataset(dir+'era_5_mys_ivtx_taiwan.nc')
ivtx=df1['p71.162']

df2=xr.open_dataset(dir+'era_5_mys_ivty_taiwan.nc')
ivty=df2['p72.162']

In [ ]:
#Q_VECTORS

#dates_start=['2005-06-11T18:00:00.000000000',
#             '2006-06-07T18:00:00.000000000',
#             '2012-06-09T18:00:00.000000000',
#             '2017-06-01T18:00:00.000000000',]

date_center=dt.datetime(2017,6,1,18)
date_start=date_center+dt.timedelta(days=-7)
date_range=[date_start+dt.timedelta(days=x) for x in range(14)]
lat_range=np.arange(-70,75,0.25)
lon_range=np.arange(50,195,0.25)  
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages(dir+"2017_event_q_vectors.pdf")
for i in range(len(date_range)):
    date=date_range[i]

    year=date.year
    month=date.month
    day=date.day
    hour=date.hour

    if month<10:
        month_str='0'+str(month)
    else:
        month_str=str(month)

    if day<10:
        day_str='0'+str(day)
    else:
        day_str=str(day)

    urlu='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_131_u.ll025uv.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'
    urlv='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_132_v.ll025uv.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'
    urlz='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_129_z.ll025sc.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'
    urlt='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_130_t.ll025sc.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'
    urlq='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_133_q.ll025sc.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'

    store_u = xr.backends.PydapDataStore.open(urlu, session=session)
    store_v = xr.backends.PydapDataStore.open(urlv, session=session)
    store_z = xr.backends.PydapDataStore.open(urlz, session=session)
    store_t = xr.backends.PydapDataStore.open(urlt, session=session)
    store_q = xr.backends.PydapDataStore.open(urlq, session=session)

    ds_u=xr.open_dataset(store_u)
    ds_v=xr.open_dataset(store_v)
    ds_z=xr.open_dataset(store_z)
    ds_t=xr.open_dataset(store_t)
    ds_q=xr.open_dataset(store_q)

    print(ds_u)

    u=ds_u.U.sel(latitude=lat_range,longitude=lon_range)
    v=ds_v.V.sel(latitude=lat_range,longitude=lon_range)
    z=ds_z.Z.sel(latitude=lat_range,longitude=lon_range)
    t=ds_t.T.sel(latitude=lat_range,longitude=lon_range)
    q=ds_q.Q.sel(latitude=lat_range,longitude=lon_range)
    
    # Grab lat/lon values

    lats = u['latitude'].metpy.unit_array
    lons = u['longitude'].metpy.unit_array

    vtime=date
    hght_850 = z.metpy.sel(vertical=700 * units.hPa,time=vtime)
    tmpk_850 = t.metpy.sel(vertical=700 * units.hPa,time=vtime)
    uwnd_850 = u.metpy.sel(vertical=700 * units.hPa,time=vtime)
    vwnd_850 = v.metpy.sel(vertical=700 * units.hPa,time=vtime)

    # Grab data and smooth using a nine-point filter applied 50 times to grab the synoptic signal
    level = 700 * units.hPa
    hght_850 = mpcalc.smooth_n_point(hght_850.squeeze(), 9, 150)
    tmpk_850 = mpcalc.smooth_n_point(tmpk_850.squeeze(), 9, 150)#25
    uwnd_850 = mpcalc.smooth_n_point(uwnd_850.squeeze(), 9, 150)
    vwnd_850 = mpcalc.smooth_n_point(vwnd_850.squeeze(), 9, 150)

    # Convert temperatures to degree Celsius for plotting purposes
    tmpc_850 = tmpk_850.to('degC')
    
    # Compute grid spacings for data
    dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

    # Compute the Q-vector components
    uqvect, vqvect = mpcalc.q_vector(uwnd_850, vwnd_850, tmpk_850, 700*units.hPa, dx, dy)

    # Compute the divergence of the Q-vectors calculated above
    q_div = -2*mpcalc.divergence(uqvect, vqvect, dx, dy, dim_order='yx')
    
    dk=xr.Dataset()
    dk['hght_850']=(('latitude','longitude'),hght_850.m)
    dk['tmpc_850']=(('latitude','longitude'),tmpc_850.m)
    dk.coords['latitude']=z.latitude
    dk.coords['longitude']=z.longitude
    hght_850=dk.hght_850
    tmpc_850=dk.tmpc_850
    hght_850=hght_850.sel(longitude=np.arange(72,180,0.25),latitude=np.arange(0,60,0.25))/100.
    tmpc_850=tmpc_850.sel(longitude=np.arange(72,180,0.25),latitude=np.arange(0,60,0.25))
    #hght_850_vals=hght_850.m/100.
    print(hght_850)
    lons1=hght_850['longitude'].metpy.unit_array
    lats1=hght_850['latitude'].metpy.unit_array
    
    # Set the map projection (how the data will be displayed)
    mapcrs = ccrs.PlateCarree(central_longitude=125)

    # Set the data project (GFS is lat/lon format)
    datacrs = ccrs.PlateCarree()

    # Start the figure and set an extent to only display a smaller graphics area
    fig = plt.figure(1, figsize=(16, 12))
    ax = plt.subplot(111, projection=mapcrs)
    ax.set_extent([85, 175, 0, 50], ccrs.PlateCarree())

    # Add map features to plot coastlines and state boundaries
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'))

    # Plot 850-hPa Q-Vector Divergence and scale
    clevs_850_tmpc = np.arange(-40, 41, 2)
    clevs_qdiv = list(range(-30, -4, 5))+list(range(5, 31, 5))
    cf = ax.contourf(lons, lats, q_div*1e18*-1., clevs_qdiv, cmap=plt.cm.bwr,
                     extend='both', transform=datacrs)
    cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True,
                      ticks=clevs_qdiv)
    cb.set_label('Q-Vector Div. (*10$^{18}$ m s$^{-1}$ kg$^{-1}$)',fontsize=20)
    cb.ax.tick_params(labelsize=18)

    # Plot 850-hPa Temperatures
    csf = ax.contour(lons1, lats1, tmpc_850, clevs_850_tmpc, colors='grey',
                     linestyles='dashed', transform=datacrs)
    plt.clabel(csf, fmt='%d',fontsize=15)

    # Plot 850-hPa Geopotential Heights
    clevs_850_hght = np.arange(0, 400, 3)
    cs = ax.contour(lons1, lats1, hght_850, clevs_850_hght, colors='black', transform=datacrs)
    plt.clabel(cs, fmt='%d',fontsize=15)

    # Plot 850-hPa Q-vectors, scale to get nice sized arrows
    wind_slice = (slice(None, None, 8), slice(None, None, 8))

    ax.quiver(lons[wind_slice[0]].magnitude, lats[wind_slice[1]].magnitude,
              uqvect[wind_slice].m,
              vqvect[wind_slice].m,
              pivot='mid', color='black',
              scale=1e-11*0.5, scale_units='inches',
              transform=datacrs)

    # Add some titles
    plt.title('700-hPa ERA5 Geo. Heights (dm), Temp (C),'
              ' and Q-Vectors (m$^2$ kg$^{-1}$ s$^{-1}$)', loc='left',fontsize=14)
    plt.title('Valid Time: {}'.format(vtime), loc='right',fontsize=14)
    #plt.title('Q-vector divergence',loc='center',fontsize=30,pad=20)
    ax.text(115,53,'Q-vector divergence',transform=ccrs.PlateCarree(),fontsize=30)
    plt.tight_layout()
    plt.show()
    
    pdf.savefig(fig)
    plt.close()
pdf.close()

In [91]:
fig.savefig(dir+'taiwan_day_plot_1a.png')

In [ ]:
#MFC

#dates_start=['2005-06-11T18:00:00.000000000',
#             '2006-06-07T18:00:00.000000000',
#             '2012-06-09T18:00:00.000000000',
#             '2017-06-01T18:00:00.000000000',]

date_center=dt.datetime(2017,6,1,18)
date_start=date_center+dt.timedelta(days=-7)
date_range=[date_start+dt.timedelta(days=x) for x in range(14)]
lat_range=np.arange(-70,75,0.25)
lon_range=np.arange(50,195,0.25)  
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages(dir+"2017_event_mfc.pdf")
for i in range(len(date_range)):
    
    date=date_range[i]
    print(date)
    vtime=date
    year=date.year
    month=date.month
    day=date.day
    hour=date.hour

    if month<10:
        month_str='0'+str(month)
    else:
        month_str=str(month)

    if day<10:
        day_str='0'+str(day)
    else:
        day_str=str(day)
        
    if month in [5,7]:
        month_length_str='31'
    if month==6:
        month_length_str='30'

    urlmfc='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.vinteg/'+str(year)+month_str+'/e5.oper.an.vinteg.162_084_viwvd.ll025sc.'+str(year)+month_str+'0100_'+str(year)+month_str+month_length_str+'23.nc'
    #https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.vinteg/201710/e5.oper.an.vinteg.162_084_viwvd.ll025sc.2017100100_2017103123.nc

    store_mfc = xr.backends.PydapDataStore.open(urlmfc, session=session)

    ds_mfc=xr.open_dataset(store_mfc)

    print(ds_mfc)

    mfc=ds_mfc.VIWVD.sel(time=date,latitude=lat_range,longitude=lon_range)
    print(mfc)
    
    lats = mfc['latitude'].metpy.unit_array
    lons = mfc['longitude'].metpy.unit_array
    # Set the map projection (how the data will be displayed)
    mapcrs = ccrs.PlateCarree(central_longitude=125)

    # Set the data project (GFS is lat/lon format)
    datacrs = ccrs.PlateCarree()

    # Start the figure and set an extent to only display a smaller graphics area
    fig = plt.figure(1, figsize=(16, 12))
    ax = plt.subplot(111, projection=mapcrs)
    ax.set_extent([85, 175, 0, 50], ccrs.PlateCarree())

    # Add map features to plot coastlines and state boundaries
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'))

    # Plot 850-hPa Q-Vector Divergence and scale
    #clevs_850_tmpc = np.arange(-40, 41, 2)
    clevs_qdiv = list(range(-30, -4, 5))+list(range(5, 31, 5))
    cf = ax.contourf(lons, lats, mfc*1e4, clevs_qdiv, cmap=plt.cm.bwr,
                     extend='both', transform=datacrs)
    cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extend='both',extendrect=True,
                      ticks=clevs_qdiv,fraction=0.046)
    cb.set_label('Moisture Flux Div. (*10$^{4}$ kg m$^{-2}$ s$^{-1}$)',fontsize=24)
    cb.ax.tick_params(labelsize=22)

    # Plot 850-hPa Temperatures
    #csf = ax.contour(lons, lats, tmpc_850, clevs_850_tmpc, colors='grey',linestyles='dashed', transform=datacrs)
    #plt.clabel(csf, fmt='%d',fontsize=14)

    # Plot 850-hPa Geopotential Heights
    #clevs_850_hght = np.arange(0, 400, 3)
    #cs = ax.contour(lons, lats, hght_850_vals, clevs_850_hght, colors='black', transform=datacrs)
    #plt.clabel(cs, fmt='%d',fontsize=15)

    # Add some titles
    plt.title('ERA5 integrated moisture flux divergence (kg m$^{-2}$ s$^{-1}$)', loc='left',fontsize=18)
    plt.title('Valid Time: {}'.format(vtime), loc='right',fontsize=18)
    #plt.title('Q-vector divergence',loc='center',fontsize=30,pad=20)
    ax.text(112,53,'Moisture flux divergence',transform=ccrs.PlateCarree(),fontsize=36)
    plt.tight_layout()
    plt.show()
    
    pdf.savefig(fig)
pdf.close()

In [93]:
fig.savefig(dir+'taiwan_day_plot_2a.png')

In [ ]:
#IVT, 850 hPa winds, TCs, detected ARs

#dates_start=['2005-06-11T18:00:00.000000000',
#             '2006-06-07T18:00:00.000000000',
#             '2012-06-09T18:00:00.000000000',
#             '2017-06-01T18:00:00.000000000',]

import matplotlib as mpl
date_center=dt.datetime(2017,6,1,18)
date_start=date_center+dt.timedelta(days=-7)
date_range=[date_start+dt.timedelta(days=x) for x in range(14)]
lat_range=np.arange(-70,75,0.25)
lon_range=np.arange(50,195,0.25)  
import matplotlib.backends.backend_pdf
#pdf = matplotlib.backends.backend_pdf.PdfPages(dir+"2017_event_ivt.pdf")

#date_range=[dt.datetime(2012,6,11,6)]
date_range=[dt.datetime(1990,6,23,6)]
for i in range(len(date_range)):
    date=date_range[i]
    vtime=date
    year=date.year
    month=date.month
    day=date.day
    hour=date.hour

    if month<10:
        month_str='0'+str(month)
    else:
        month_str=str(month)

    if day<10:
        day_str='0'+str(day)
    else:
        day_str=str(day)

    urlu='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_131_u.ll025uv.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'
    urlv='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_132_v.ll025uv.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'
    urlz='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_129_z.ll025sc.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'
    urlt='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_130_t.ll025sc.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'
    urlq='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_133_q.ll025sc.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'

    store_u = xr.backends.PydapDataStore.open(urlu, session=session)
    store_v = xr.backends.PydapDataStore.open(urlv, session=session)
    store_z = xr.backends.PydapDataStore.open(urlz, session=session)
    store_t = xr.backends.PydapDataStore.open(urlt, session=session)
    store_q = xr.backends.PydapDataStore.open(urlq, session=session)

    ds_u=xr.open_dataset(store_u)
    ds_v=xr.open_dataset(store_v)
    ds_z=xr.open_dataset(store_z)
    ds_t=xr.open_dataset(store_t)
    ds_q=xr.open_dataset(store_q)

    print(ds_u)

    u=ds_u.U.sel(latitude=lat_range,longitude=lon_range)
    v=ds_v.V.sel(latitude=lat_range,longitude=lon_range)
    z=ds_z.Z.sel(latitude=lat_range,longitude=lon_range)
    t=ds_t.T.sel(latitude=lat_range,longitude=lon_range)
    q=ds_q.Q.sel(latitude=lat_range,longitude=lon_range)
    
    # Grab lat/lon values

    lats = u['latitude'].metpy.unit_array
    lons = u['longitude'].metpy.unit_array

    vtime=date
    hght_850 = z.metpy.sel(vertical=700 * units.hPa,time=vtime)
    tmpk_850 = t.metpy.sel(vertical=700 * units.hPa,time=vtime)
    uwnd_850 = u.metpy.sel(vertical=700 * units.hPa,time=vtime)
    vwnd_850 = v.metpy.sel(vertical=700 * units.hPa,time=vtime)

    # Grab data and smooth using a nine-point filter applied 50 times to grab the synoptic signal
    level = 700 * units.hPa
    hght_850 = mpcalc.smooth_n_point(hght_850.squeeze(), 9, 150)
    tmpk_850 = mpcalc.smooth_n_point(tmpk_850.squeeze(), 9, 150)#25
    uwnd_850 = mpcalc.smooth_n_point(uwnd_850.squeeze(), 9, 150)
    vwnd_850 = mpcalc.smooth_n_point(vwnd_850.squeeze(), 9, 150)

    # Convert temperatures to degree Celsius for plotting purposes
    tmpc_850 = tmpk_850.to('degC')
    
    # Compute grid spacings for data
    dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

    # Compute the Q-vector components
    uqvect, vqvect = mpcalc.q_vector(uwnd_850, vwnd_850, tmpk_850, 700*units.hPa, dx, dy)

    # Compute the divergence of the Q-vectors calculated above
    q_div = -2*mpcalc.divergence(uqvect, vqvect, dx, dy, dim_order='yx')
    
    dk=xr.Dataset()
    dk['hght_850']=(('latitude','longitude'),hght_850.m)
    dk['tmpc_850']=(('latitude','longitude'),tmpc_850.m)
    dk.coords['latitude']=z.latitude
    dk.coords['longitude']=z.longitude
    hght_850=dk.hght_850
    tmpc_850=dk.tmpc_850
    hght_850=hght_850.sel(longitude=np.arange(72,180,0.25),latitude=np.arange(0,60,0.25))/100.
    tmpc_850=tmpc_850.sel(longitude=np.arange(72,180,0.25),latitude=np.arange(0,60,0.25))
    #hght_850_vals=hght_850.m/100.
    print(hght_850)
    lons1=hght_850['longitude'].metpy.unit_array
    lats1=hght_850['latitude'].metpy.unit_array
    mapcrs = ccrs.PlateCarree(central_longitude=125)
    count_fig=0
    count_file=0

    if 1==1:
        print(date)
        #date=pd.to_datetime(date)
        
        year=date.year
        month=date.month
        day=date.day
        hour=date.hour
        
        
        ivtx_current=ivtx.sel(time=date)
        ivty_current=ivty.sel(time=date)
        ivt_mag_current=np.sqrt(ivtx_current**2+ivty_current**2)

        u_850_select=u.sel(time=date,level=850)
        v_850_select=v.sel(time=date,level=850)
        slp_select=slp.sel(time=date)

        ivt_250=ivt_mag_current.where(ivt_mag_current>=250.)
        
        ar_select=ar.sel(time=date)
        
        if 1==1:
            fig = plt.figure(1, figsize=(16, 12))
            ax = plt.subplot(111, projection=mapcrs)
            ax.set_extent([85, 175, 0, 50], ccrs.PlateCarree())
        
        check_lat_tc=lat_tc.where(years_tc==year,drop=True)
        check_lat_tc=check_lat_tc.where(months_tc==month,drop=True)
        check_lat_tc=check_lat_tc.where(days_tc==day,drop=True)
        check_lat_tc=check_lat_tc.where(hours_tc==hour,drop=True)
        #TS,SS,EX,ET,
        check_lat_tc_trop=check_lat_tc.where(nature_tc==1,drop=True)
        check_lat_tc_nontrop=check_lat_tc.where(nature_tc!=1,drop=True)
        
        check_lat_tc_trop_list=[]
        for i in range(check_lat_tc_trop.storm.size):
            check_lat_tc_trop_values=check_lat_tc_trop[i,:].values.tolist()
            check_lat_tc_trop_values=[x for x in check_lat_tc_trop_values if x>0]
            if len(check_lat_tc_trop_values)>0:
                check_lat_tc_trop_value=check_lat_tc_trop_values[0]
                check_lat_tc_trop_list.append(check_lat_tc_trop_value)
                
        check_lat_tc_nontrop_list=[]
        for i in range(check_lat_tc_nontrop.storm.size):
            check_lat_tc_nontrop_values=check_lat_tc_nontrop[i,:].values.tolist()
            check_lat_tc_nontrop_values=[x for x in check_lat_tc_nontrop_values if x>0]
            if len(check_lat_tc_nontrop_values)>0:
                check_lat_tc_nontrop_value=check_lat_tc_nontrop_values[0]
                check_lat_tc_nontrop_list.append(check_lat_tc_nontrop_value)
        
        check_lon_tc=lon_tc.where(years_tc==year,drop=True)
        check_lon_tc=check_lon_tc.where(months_tc==month,drop=True)
        check_lon_tc=check_lon_tc.where(days_tc==day,drop=True)
        check_lon_tc=check_lon_tc.where(hours_tc==hour,drop=True)
        check_lon_tc_trop=check_lon_tc.where(nature_tc==1,drop=True)
        check_lon_tc_nontrop=check_lon_tc.where(nature_tc!=1,drop=True)
        
        check_lon_tc_trop_list=[]
        for i in range(check_lon_tc_trop.storm.size):
            check_lon_tc_trop_values=check_lon_tc_trop[i,:].values.tolist()
            check_lon_tc_trop_values=[x for x in check_lon_tc_trop_values if x>-1000]
            if len(check_lon_tc_trop_values)>0:
                check_lon_tc_trop_value=check_lon_tc_trop_values[0]
                check_lon_tc_trop_list.append(check_lon_tc_trop_value)
                
        check_lon_tc_nontrop_list=[]
        for i in range(check_lon_tc_nontrop.storm.size):
            check_lon_tc_nontrop_values=check_lon_tc_nontrop[i,:].values.tolist()
            check_lon_tc_nontrop_values=[x for x in check_lon_tc_nontrop_values if x>-1000]
            if len(check_lon_tc_nontrop_values)>0:
                check_lon_tc_nontrop_value=check_lon_tc_nontrop_values[0]
                check_lon_tc_nontrop_list.append(check_lon_tc_nontrop_value)
        
        if len(check_lat_tc_trop_list)>0 and len(check_lon_tc_trop_list)>0:
            for i in range(len(check_lat_tc_trop_list)):
                lat_tc_single=check_lat_tc_trop_list[i]
                lon_tc_single=check_lon_tc_trop_list[i]
                print((lat_tc_single,lon_tc_single))
                if 0<=lat_tc_single<=50 and 85<=lon_tc_single<=175:
                    ax.plot(lon_tc_single,lat_tc_single,marker='*',color='r',transform=ccrs.PlateCarree(),markersize=18,zorder=41)
                
        if len(check_lat_tc_nontrop_list)>0 and len(check_lon_tc_nontrop_list)>0:
            for i in range(len(check_lat_tc_nontrop_list)):
                lat_tc_single=check_lat_tc_nontrop_list[i]
                lon_tc_single=check_lon_tc_nontrop_list[i]
                print((lat_tc_single,lon_tc_single))
                if 0<=lat_tc_single<=50 and 85<=lon_tc_single<=175:
                    ax.plot(lon_tc_single,lat_tc_single,marker='*',color='saddlebrown',transform=ccrs.PlateCarree(),markersize=18,zorder=41)
                
        if 1==1:             
            ax.coastlines(resolution='10m')
            #ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
            #ax.add_feature(cfeature.LAKES.with_scale('50m'))
            countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
            ax.add_feature(countries)
            
            if 1==0:
                # *must* call draw in order to get the axis boundary used to add ticks:
                fig.canvas.draw()

                # Define gridline locations and draw the lines using cartopy's built-in gridliner:
                xticks = [-135,-125,-115,-105,-95,-85,-75,-65,-55,-45,-35,-25]
                yticks = [5,10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
                ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0.5)
                ax.tick_params(labelsize=24)
                # Label the end-points of the gridlines using the custom tick makers:
                ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
                ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
                lambert_xticks(ax, xticks)
                lambert_yticks(ax, yticks)

            cax1=ax.contourf(ivtx.longitude,ivtx.latitude,ivt_250,levels=np.arange(250,1040,50),extend='both',transform=ccrs.PlateCarree(),cmap=plt.cm.Greys,alpha=0.5)
            
            cbar = plt.colorbar(cax1, orientation='horizontal', pad=0.003, aspect=50,fraction=0.046)
            cbar.set_label('IVT (kg m$^{-1}$ s$^{-1}$)',fontsize=20,labelpad=5)
            cbar.ax.tick_params(labelsize=18)

            c=ax.contour(slp_select.longitude,slp_select.latitude,slp_select/100.,linewidths=1.0,transform=ccrs.PlateCarree(),colors='k',levels=np.arange(960,1040,4))
            plt.clabel(c,fmt='%.00f',fontsize=15)
            X=8
            q=ax.quiver(u_850_select.longitude[0::X].values,u_850_select.latitude[0::X].values,u_850_select[0::X,0::X].values,v_850_select[0::X,0::X].values,transform=ccrs.PlateCarree(),units='inches',scale=30,width=0.025,alpha=0.5)
            ax.quiverkey(q,0.67,1.01,10,'10 m/s',fontproperties={'size':13})

            #cax21=ax.contourf(ivtx.longitude,ivtx.latitude,ivt_250.where(ar_select==1),levels=np.arange(250,1040,50),extend='both',transform=ccrs.PlateCarree(),cmap=plt.cm.Greys,alpha=0.5,label='Identified AR')
            cax2=ax.contourf(ar_select.longitude,ar_select.latitude,ar_select,[0,1],colors='b',transform=ccrs.PlateCarree(),hatches=[None,'.'],alpha=0.1,label='Identified AR',zorder=20)

            mpl.rcParams['hatch.linewidth']=0.01
            vtime=date
            plt.title('ERA5 850-hPa winds (m/s), MSLP (hPa), and IVT (kg m$^{-1}$ s$^{-1}$)',loc='left',fontsize=14)
            plt.title('Valid Time: {}'.format(vtime), loc='right',fontsize=14)
            #ax.text(120,53,'AR detection',transform=ccrs.PlateCarree(),fontsize=30)
            ax.text(120,53,'TC-dominant',transform=ccrs.PlateCarree(),fontsize=30)
            plt.tight_layout()
            plt.show()
            fig.savefig(dir+'taiwan_paper_examples_2.png')
        #pdf.savefig(fig)
#pdf.close()

In [131]:
fig.savefig(dir+'taiwan_day_plot_3a.png')

In [ ]:
for i in range(41):#41; 60
    year=1979+i
    #year=1960+i
    print(year)
    base=dt.datetime(year,5,15,6)#middle of first day (starts 18Z on 14th)
    dates_list_mys=[base+dt.timedelta(days=x) for x in range(47)]
    ds=xr.open_dataset(dir_rain+'rain.'+str(year)+'.daily.1km-grid-v2.nc')
    p=ds.rain
    dates=[]
    for j in range(len(ds.time)):
        dates.append(dt.datetime(year,1,1,6)+dt.timedelta(j))
    p['time']=dates#now have datetime objects
    p_mys=p.sel(time=dates_list_mys)
    
    if i==0:
        p_mys_all=p_mys
    else:
        p_mys_all=xr.concat([p_mys_all,p_mys],dim='time')

In [ ]:
#TAIWAN_PRECIP

#This script plots composite of Taiwan EP days
#dates_start=['2005-06-11T18:00:00.000000000',
#             '2006-06-07T18:00:00.000000000',
#             '2012-06-09T18:00:00.000000000',
#             '2017-06-01T18:00:00.000000000',]

date_center=dt.datetime(2017,6,1,18)
date_midday=date_center+dt.timedelta(hours=12)
date_start=date_midday+dt.timedelta(days=-7)
date_range=[date_start+dt.timedelta(days=x) for x in range(14)]
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages(dir+"2017_event_rain.pdf")
for i in range(len(date_range)):
    date=date_range[i]
    print(date)
    vtime=date
    p_select=p_mys_all.sel(time=date)

    fig = plt.figure(figsize=(13,12))
    ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
    ax.set_extent([118.38,123.77,21.73,25.45], ccrs.PlateCarree())
    cax=ax.contourf(p.lon,p.lat,p_select,levels=np.arange(0,310,10),extend='max',cmap=plt.cm.Greens)
    cbar=plt.colorbar(cax,pad=0,orientation='horizontal',aspect=50,ticks=[40,80,120,160,200,240,280])
    cbar.set_label('Precipitation (mm/day)',fontsize=20)
    cbar.ax.tick_params(labelsize=18)
    dy1=xr.open_dataset(dir2+'etopo1-2.nc')
    elevation=dy1['Band1']
    X=6
    elevation_deres=xr.zeros_like(elevation[0::X,0::X])
    elevation_deres['lat']=[x+float(X)/2./111. for x in elevation_deres.lat.values]
    elevation_deres['lon']=[x+float(X)/2./102. for x in elevation_deres.lon.values]
    for i in range(int(elevation.lat.size/X)):
        for j in range(int(elevation.lon.size/X)):
            elevation_block=elevation[X*i:X*(i+1),X*j:X*(j+1)]
            elevation_mean=elevation_block.mean(dim=('lat','lon')).values
            elevation_deres[i,j]=elevation_mean
    elevation_deres=elevation_deres.where(elevation_deres>-200)
    ax.contour(elevation_deres.lon,elevation_deres.lat,elevation_deres,levels=[400,1200,2000,2800,3600,4400],colors='k',linewidths=[0.8,1.0,1.4,1.6,1.8,2.0],zorder=20,alpha=1.0)
    ax.coastlines(resolution='10m')


    ax.set_title('TCCIP 1-km gridded precip',loc='left',fontsize=14)
    plt.title('Valid Time: {}'.format(vtime), loc='right',fontsize=14)
    ax.text(120.3,25.6,'Daily precipitation',fontsize=30)
    plt.tight_layout()
    plt.show()
    pdf.savefig(fig)
pdf.close()

In [126]:
fig.savefig(dir+'taiwan_day_plot_4a.png')

In [ ]:
#250 hPa wind speed; 1000-500 hPa thickness; MSLP

#This script plots composite of Taiwan EP days
#dates_start=['2005-06-11T18:00:00.000000000',
#             '2006-06-07T18:00:00.000000000',
#             '2012-06-09T18:00:00.000000000',
#             '2017-06-01T18:00:00.000000000',]

mapcrs = ccrs.PlateCarree(central_longitude=125)
date_center=dt.datetime(2017,6,1,18)
date_start=date_center+dt.timedelta(days=-7)
date_range=[date_start+dt.timedelta(days=x) for x in range(14)]
lat_range=np.arange(-70,75,0.25)
lon_range=np.arange(50,195,0.25)  
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages(dir+"2017_event_upperlevel.pdf")
for i in range(len(date_range)):
    date=date_range[i]

    year=date.year
    month=date.month
    day=date.day
    hour=date.hour

    if month<10:
        month_str='0'+str(month)
    else:
        month_str=str(month)

    if day<10:
        day_str='0'+str(day)
    else:
        day_str=str(day)

    urlu='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_131_u.ll025uv.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'
    urlv='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_132_v.ll025uv.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'
    urlz='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_129_z.ll025sc.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'
    urlt='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_130_t.ll025sc.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'
    urlq='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'+str(year)+month_str+'/e5.oper.an.pl.128_133_q.ll025sc.'+str(year)+month_str+day_str+'00_'+str(year)+month_str+day_str+'23.nc'

    store_u = xr.backends.PydapDataStore.open(urlu, session=session)
    store_v = xr.backends.PydapDataStore.open(urlv, session=session)
    store_z = xr.backends.PydapDataStore.open(urlz, session=session)
    store_t = xr.backends.PydapDataStore.open(urlt, session=session)
    store_q = xr.backends.PydapDataStore.open(urlq, session=session)

    ds_u=xr.open_dataset(store_u)
    ds_v=xr.open_dataset(store_v)
    ds_z=xr.open_dataset(store_z)
    ds_t=xr.open_dataset(store_t)
    ds_q=xr.open_dataset(store_q)

    print(ds_u)

    u=ds_u.U.sel(latitude=lat_range,longitude=lon_range)
    v=ds_v.V.sel(latitude=lat_range,longitude=lon_range)
    z=ds_z.Z.sel(latitude=lat_range,longitude=lon_range)
    t=ds_t.T.sel(latitude=lat_range,longitude=lon_range)
    q=ds_q.Q.sel(latitude=lat_range,longitude=lon_range)
    
    # Grab lat/lon values

    lats = u['latitude'].metpy.unit_array
    lons = u['longitude'].metpy.unit_array

    vtime=date
    hght_850 = z.metpy.sel(vertical=700 * units.hPa,time=vtime)
    tmpk_850 = t.metpy.sel(vertical=700 * units.hPa,time=vtime)
    uwnd_850 = u.metpy.sel(vertical=700 * units.hPa,time=vtime)
    vwnd_850 = v.metpy.sel(vertical=700 * units.hPa,time=vtime)

    # Grab data and smooth using a nine-point filter applied 50 times to grab the synoptic signal
    level = 700 * units.hPa
    hght_850 = mpcalc.smooth_n_point(hght_850.squeeze(), 9, 150)
    tmpk_850 = mpcalc.smooth_n_point(tmpk_850.squeeze(), 9, 150)#25
    uwnd_850 = mpcalc.smooth_n_point(uwnd_850.squeeze(), 9, 150)
    vwnd_850 = mpcalc.smooth_n_point(vwnd_850.squeeze(), 9, 150)

    # Convert temperatures to degree Celsius for plotting purposes
    tmpc_850 = tmpk_850.to('degC')
    
    # Compute grid spacings for data
    dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

    # Compute the Q-vector components
    uqvect, vqvect = mpcalc.q_vector(uwnd_850, vwnd_850, tmpk_850, 700*units.hPa, dx, dy)

    # Compute the divergence of the Q-vectors calculated above
    q_div = -2*mpcalc.divergence(uqvect, vqvect, dx, dy, dim_order='yx')
    
    dk=xr.Dataset()
    dk['hght_850']=(('latitude','longitude'),hght_850.m)
    dk['tmpc_850']=(('latitude','longitude'),tmpc_850.m)
    dk.coords['latitude']=z.latitude
    dk.coords['longitude']=z.longitude
    hght_850=dk.hght_850
    tmpc_850=dk.tmpc_850
    hght_850=hght_850.sel(longitude=np.arange(72,180,0.25),latitude=np.arange(0,60,0.25))/100.
    tmpc_850=tmpc_850.sel(longitude=np.arange(72,180,0.25),latitude=np.arange(0,60,0.25))
    #hght_850_vals=hght_850.m/100.
    print(hght_850)
    lons1=hght_850['longitude'].metpy.unit_array
    lats1=hght_850['latitude'].metpy.unit_array

    vtime=date
    hght_250 = z.metpy.sel(vertical=250 * units.hPa,time=vtime)
    tmpk_250 = t.metpy.sel(vertical=250 * units.hPa,time=vtime)
    uwnd_250 = u.metpy.sel(vertical=250 * units.hPa,time=vtime)
    vwnd_250 = v.metpy.sel(vertical=250 * units.hPa,time=vtime)
    speed_250=np.sqrt(uwnd_250**2+vwnd_250**2)
    slp_select=slp.sel(time=date)
    thickness=z.sel(time=date,level=500)-z.sel(time=date,level=1000)
    print(speed_250.max().values)

    if 1==1:
        if 1==1:
            fig = plt.figure(figsize=(16,12))
            ax=plt.subplot(1,1,1,projection=mapcrs)
            ax.coastlines(resolution='10m')
            countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
            ax.add_feature(countries)
            ax.set_extent([85, 175, 0, 50], ccrs.PlateCarree())
            
            if 1==0:
                # *must* call draw in order to get the axis boundary used to add ticks:
                fig.canvas.draw()

                # Define gridline locations and draw the lines using cartopy's built-in gridliner:
                xticks = [-135,-125,-115,-105,-95,-85,-75,-65,-55,-45,-35,-25]
                yticks = [5,10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
                ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0.5)
                ax.tick_params(labelsize=24)
                # Label the end-points of the gridlines using the custom tick makers:
                ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
                ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
                lambert_xticks(ax, xticks)
                lambert_yticks(ax, yticks)

            #cax=ax.scatter(lons_extreme_list,lats_extreme_list,s=150,c=obs_extreme_list,transform=ccrs.PlateCarree(),vmin=0,vmax=120,cmap=plt.cm.RdYlGn_r,zorder=10)
            cax1=ax.contourf(speed_250.longitude,speed_250.latitude,speed_250.where(speed_250>=30.),levels=np.arange(30,120,10),transform=ccrs.PlateCarree(),colors=['skyblue','deepskyblue','dodgerblue','lightpink','lightcoral','tomato','deeppink','lightgray','whitesmoke'],alpha=0.5)
            
            cbar = plt.colorbar(cax1, orientation='horizontal', pad=0, aspect=50,fraction=0.046,ticks=[40,50,60,70,80,90,100])
            cbar.set_label('m/s',fontsize=24,labelpad=10)
            cbar.ax.tick_params(labelsize=22)

            c=ax.contour(slp_select.longitude,slp_select.latitude,slp_select/100.,linewidths=1.5,transform=ccrs.PlateCarree(),colors='k',levels=np.arange(960,1040,4))
            plt.clabel(c,fmt='%.00f',fontsize=15)
            
            c1=ax.contour(thickness.longitude,thickness.latitude,thickness/100.,linewidths=1.5,linestyles='--',transform=ccrs.PlateCarree(),colors=['b','b','b','b','b','r','r','r','r','r','r'],levels=np.arange(516,580,6))
            plt.clabel(c1,fmt='%.00f',fontsize=15)
            
            #ax.set_title(str(hour)+'Z '+month_dict[str(date.month)]+' '+str(date.day)+', '+str(date.year),fontsize=36)
            
            vtime=date
            plt.title('ERA5 250-hPa wind speed (m/s), MSLP (hPa), and 1000-500-hPa thickness (dm)',loc='left',fontsize=17)
            plt.title('Valid Time: {}'.format(vtime), loc='right',fontsize=17)
            #plt.title('Q-vector divergence',loc='center',fontsize=30,pad=20)
            ax.text(120,53,'Upper-level',transform=ccrs.PlateCarree(),fontsize=36)
            plt.tight_layout()
            plt.show()
    pdf.savefig(fig)
pdf.close()